In [13]:
import json
import torch
from gensim.models import KeyedVectors
from gensim import matutils
import numpy as np

In [14]:
print('Loading Vocabulary and Vectors')
word2vec = KeyedVectors.load_word2vec_format(
    'data/word2vec/GoogleNews-vectors-negative300.bin', binary=True)
vocabulary = json.load(open('data/processed_data/processed_data.json', 'r'))

Loading Vocabulary and Vectors


In [51]:
scores = rouge.get_scores(
    generated_summary, reference_summary
)
print(scores)

tensor(1.6194, grad_fn=<MseLossBackward0>)

In [102]:
def word2vec_emb(sequence, word2vec, vocabulary):
    
    seq_tokens = []
    
    for key in sequence:
        try:
            seq_tokens.append(vocabulary['ind2word'][str(key.item())])
        except:
            seq_tokens.append(vocabulary['ind2word'][list(
                vocabulary['ind2word'].keys())[-1]])
    
    vectorized = []
    for word in seq_tokens:
        try:
            vectorized.append(word2vec[word])
        except:
            vectorized.append(np.zeros(300,))
    
    return np.array(vectorized)

In [103]:
def similarity_cosine(vec1, vec2):
    cosine_similarity = np.dot(matutils.unitvec(vec1), matutils.unitvec(vec2))
    print(cosine_similarity)
    print(spatial.distance.cosine(vec1, vec2))
    return cosine_similarity

In [104]:
def reward(target, generated, word2vec, vocabulary):
    '''
    Calculate the reward for the generated text
    '''
    target = word2vec_emb(target, word2vec, vocabulary)
    generated = word2vec_emb(generated, word2vec, vocabulary)
    
    reward = similarity_cosine(target.mean(axis=0), generated.mean(axis=0))

    return reward

In [123]:
answer = torch.tensor([[6100, 4948, 6066, 1336, 4264, 4093,   37, 3343, 2467, 5527, 3396, 4326,
          219, 5560, 5845, 1464, 3149, 2805, 4154, 1899, 5642]])

question = torch.tensor([[6100, 2440, 5128, 5570,  511, 2483, 4828, 3564,  282, 3869, 2614,    7,
         4783, 4595, 5613, 1385, 6032, 4369,  286, 2259, 1900]])

summary = torch.tensor([[6100, 5955, 2642, 6021, 1982, 3081, 3554, 3802, 4577, 5298, 5555, 1110,
         4515, 2661, 2299, 4606, 4956, 4007, 3281, 2625, 2206, 4335, 2730, 2925,
         4523, 1231, 4269,  427,  861, 4104, 2630, 2499, 5430, 4043, 5146, 4595,
         3772, 5545, 4852, 3325, 4324]])

gtSummary = torch.tensor([[405,  333,  880,  188,  624,  194,  996,  293,  244, 6099, 3275,   99,
           25,  244,  754,  636,  936, 1461, 3653,   51,   55,   56,   57,    1,
           58,    1,   59,    1,   60,   52,  244, 1367, 6099, 1368, 3654, 2424,
           76,   51, 3655,   52]])

In [159]:
from scipy import spatial

In [128]:
# torch.cat((answer[0], question[0]))
print(reward(target=torch.cat((answer[0], question[0])), generated=summary[0], word2vec=word2vec, vocabulary=vocabulary))
print(reward(target=gtSummary[0], generated=summary[0], word2vec=word2vec, vocabulary=vocabulary))

0.7565139467498903
0.6523351061715779


In [1]:
from rouge import Rouge
def rouge_scores(target, generated, word2vec, vocabulary):
    rouge = Rouge()
    tar = []
    ref = []

    for key in target:
        try:
            if vocabulary['ind2word'][str(key.item())] !='UNK':
                tar.append(vocabulary['ind2word'][str(key.item())])
        except:
            pass

    for key in generated:
        try:
            if vocabulary['ind2word'][str(key.item())] !='UNK':
                ref.append(vocabulary['ind2word'][str(key.item())])
        except:
            pass

    return rouge.get_scores(' '.join(ref), ' '.join(tar), avg=True)

In [158]:
rouge_scores(torch.cat((answer[0], question[0])), summary[0], word2vec, vocabulary)

40 visible one-half calling retinal open-angle missing elbows street hopeless absolutely S loss Depakene TEE acid minimal double recurrences opposite Cutting tends efforts settings hepatitis reflexes heart-lung Name 1 Oxygen endoscopy if Autonomic beat companies cervical Nonne-Milroy separates Fragile osteochondrodysplasias abnormally
- - - - - - 
40 running electrical Recurrent barley amyloid Rapid senses containers Evidence Hospitalization testing Joseph FODMAP x-rays fasting coated Southeast psoralen Sharing natural G pacing mouthwashes Rasagiline While psychiatric resistance new lights relieving babies Involved specially firm beat relationships instability cookies colloidal mechanisms


{'rouge-1': {'r': 0.025, 'p': 0.025, 'f': 0.024999995000001003},
 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-l': {'r': 0.025, 'p': 0.025, 'f': 0.024999995000001003}}

In [138]:
Rouge().get_scores('I go to the beach','I go to the beach every data', avg=True)

{'rouge-1': {'r': 0.7142857142857143, 'p': 1.0, 'f': 0.8333333284722222},
 'rouge-2': {'r': 0.6666666666666666, 'p': 1.0, 'f': 0.7999999952000001},
 'rouge-l': {'r': 0.7142857142857143, 'p': 1.0, 'f': 0.8333333284722222}}

In [242]:
import os
import json
import random
import numpy as np
from six.moves import range

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision

import lc_options
from utils import lc_utilities as utils
from rouge import Rouge

In [244]:
params = {
    'inputJson': "data/visdial/chat_processed_params.json",
    'useGPU': False,
    
    # A-Bot checkpoint
    'startFrom': "./checkpoints/vis_a/abot_ep_1.vd",
    
    # Q-Bot checkpoint should given if interactive dialog is required
    'qstartFrom': "./checkpoints/vis_q/qbot_ep_0.vd",
    
    'beamSize': 5,
}

# RNG seed
manualSeed = 1597
random.seed(manualSeed)
torch.manual_seed(manualSeed)
if params['useGPU']:
    torch.cuda.manual_seed_all(manualSeed)

print('Loading json file: ' + params['inputJson'])
with open(params['inputJson'], 'r') as fileId:
    info = json.load(fileId)

wordCount = len(info['word2ind'])
# Add <START> and <END> to vocabulary
info['word2ind']['<START>'] = wordCount + 1
info['word2ind']['<END>'] = wordCount + 2
startToken = info['word2ind']['<START>']
endToken = info['word2ind']['<END>']
# Padding token is at index 0
vocabSize = wordCount + 3
print('Vocab size with <START>, <END>: %d' % vocabSize)

# Construct the reverse map
info['ind2word'] = {
    int(ind): word
    for word, ind in info['word2ind'].items()
}


Loading json file: data/visdial/chat_processed_params.json
Vocab size with <START>, <END>: 8333


In [245]:

def loadModel(params, agent='abot'):
    # should be everything used in encoderParam, decoderParam below
    encoderOptions = [
        'encoder', 'vocabSize', 'embedSize', 'rnnHiddenSize', 'numLayers',
        'useHistory', 'numRounds', 'dropout', 'useSumm'
    ]
    decoderOptions = [
        'decoder', 'vocabSize', 'embedSize', 'rnnHiddenSize', 'numLayers',
        'dropout'
    ]
    modelOptions = encoderOptions + decoderOptions

    mdict = None
    gpuFlag = params['useGPU']
    startArg = 'startFrom' if agent == 'abot' else 'qstartFrom'
    assert params[startArg], "Need checkpoint for {}".format(agent)

    if params[startArg]:
        print('Loading model (weights and config) from {}'.format(
            params[startArg]))

        if gpuFlag:
            mdict = torch.load(params[startArg])
        else:
            mdict = torch.load(params[startArg],
                map_location=lambda storage, location: storage)

        # Model options is a union of standard model options defined
        # above and parameters loaded from checkpoint
        modelOptions = list(set(modelOptions).union(set(mdict['params'])))
        for opt in modelOptions:
            if opt not in params:
                params[opt] = mdict['params'][opt]

            elif params[opt] != mdict['params'][opt]:
                # Parameters are not overwritten from checkpoint
                pass

    # Initialize model class
    encoderParam = {k: params[k] for k in encoderOptions}
    decoderParam = {k: params[k] for k in decoderOptions}

    encoderParam['startToken'] = encoderParam['vocabSize'] - 2
    encoderParam['endToken'] = encoderParam['vocabSize'] - 1
    decoderParam['startToken'] = decoderParam['vocabSize'] - 2
    decoderParam['endToken'] = decoderParam['vocabSize'] - 1

    if agent == 'abot':
        encoderParam['type'] = params['encoder']
        decoderParam['type'] = params['decoder']
        encoderParam['isAnswerer'] = True
        from lc.models.lc_answerer import Answerer
        model = Answerer(encoderParam, decoderParam)

    elif agent == 'qbot':
        encoderParam['type'] = params['qencoder']
        decoderParam['type'] = params['qdecoder']
        encoderParam['isAnswerer'] = False
        encoderParam['useSumm'] = False
        from lc.models.lc_questioner import Questioner
        model = Questioner(
            encoderParam,
            decoderParam,
            summGenSize=60)

    # if params['useGPU']:
    #     model.cuda()

    if mdict:
        model.load_state_dict(mdict['model'])
        
    print("Loaded agent {}".format(agent))
    return model

In [246]:
aBot = None
qBot = None



In [247]:
# load aBot
if params['startFrom']:
    aBot = loadModel(params, 'abot')
    assert aBot.encoder.vocabSize == vocabSize, "Vocab size mismatch!"
    aBot.eval()
    
# load qBot
if params['qstartFrom']:
    qBot = loadModel(params, 'qbot')
    assert qBot.encoder.vocabSize == vocabSize, "Vocab size mismatch!"
    qBot.eval()

Loading model (weights and config) from ./checkpoints/vis_a/abot_ep_1.vd
Encoder: hre-ques-lateim-hist
Decoder: gen
Loaded agent abot
Loading model (weights and config) from ./checkpoints/vis_q/qbot_ep_0.vd
Encoder: hre-ques-lateim-hist
Decoder: gen
Loaded agent qbot


In [248]:
ind_map = lambda words: np.array([info['word2ind'].get(word, info['word2ind']['UNK']) 
                                  for word in words], dtype='int64')

tokenize = lambda string: ['<START>'] + word_tokenize(string) + ['<END>']

In [249]:
# Helper functions for converting tensors to words
to_str_pred = lambda w, l: str(" ".join([info['ind2word'][x] for x in list( filter(
        lambda x:x>0,w.data.cpu().numpy()))][:l.data.cpu().item()]))[8:]
to_str_gt = lambda w: str(" ".join([info['ind2word'][x] for x in filter(
        lambda x:x>0,w.data.cpu().numpy())]))[8:-6]

In [250]:
def var_map(tensor):
    return Variable(tensor.unsqueeze(0), volatile=True)

In [257]:
doc = "A vase of flowers sitting on table"
docs = doc
summary = 'A vase of flowers sitting on table'

words = nltk.word_tokenize(doc)
words = [word.lower() for word in words if word.isalpha()]
doc = ' '.join(words)

words = nltk.word_tokenize(summary)
words = [word.lower() for word in words if word.isalpha()]
summary = ' '.join(words)





# Process document
doc_tokens = tokenize(doc)
doc = ind_map(doc_tokens)

# Process document
summ_tokens = tokenize(summary)
summ = ind_map(summ_tokens)


doc_tensor = var_map(torch.from_numpy(doc))
doc_lens = var_map(torch.LongTensor([len(doc)]))

summ_tensor = var_map(torch.from_numpy(summ))
summ_lens = var_map(torch.LongTensor([len(summ)]))

C:\Users\anest\AppData\Local\Temp\ipykernel_33656\881476409.py:2: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(tensor.unsqueeze(0), volatile=True)


In [258]:
q = "how is hemophilia treated ?"

words = nltk.word_tokenize(q)
words = [word.lower() for word in words if word.isalpha()]
q = ' '.join(words)

q_tokens = tokenize(q)
q = ind_map(q_tokens)

q_tensor = var_map(torch.from_numpy(q))
q_lens = var_map(torch.LongTensor([len(q)]))

C:\Users\anest\AppData\Local\Temp\ipykernel_33656\881476409.py:2: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(tensor.unsqueeze(0), volatile=True)


In [259]:
if aBot:
     aBot.eval(), aBot.reset()
     aBot.train(), aBot.reset()
     aBot.observe(-1, summary=summ_tensor, summaryLens=summ_lens, document=doc_tensor,
                    documentLens=doc_lens)
    
     aBot.observe(0, ques=q_tensor, quesLens=q_lens)
     answers, ansLens = aBot.forwardDecode(
         inference='greedy', beamSize=15)
     aBot.observe(0, ans=answers, ansLens=ansLens)
     print("A%d: "%(0+1), to_str_gt(answers[0]))
     
     aBot.observe(1, ques=q_tensor, quesLens=q_lens)
     answers, ansLens = aBot.forwardDecode(
         inference='greedy', beamSize=15)
     aBot.observe(1, ans=answers, ansLens=ansLens)
     print("A%d: "%(0+1), to_str_gt(answers[0]))
     
     aBot.observe(2, ques=q_tensor, quesLens=q_lens)
     answers, ansLens = aBot.forwardDecode(
         inference='greedy', beamSize=15)
     aBot.observe(2, ans=answers, ansLens=ansLens)
     print("A%d: "%(0+1), to_str_gt(answers[0]))

A1:  white <END> <END>
A1:  white <END>
A1:  white <END>


In [264]:
if aBot:
    aBot.eval(), aBot.reset()
    aBot.train(), aBot.reset()
    aBot.observe(-1, summary=summ_tensor, summaryLens=summ_lens, document=doc_tensor,
                    documentLens=doc_lens)

if qBot:
    qBot.eval(), qBot.reset()
    qBot.observe(-1, document=doc_tensor,
                    documentLens=doc_lens)

numRounds = 10
beamSize = 5

for round in range(numRounds):
    questions, quesLens = qBot.forwardDecode(
         inference='greedy', beamSize=beamSize)
    qBot.observe(round, ques=questions, quesLens=quesLens)
    aBot.observe(round, ques=questions, quesLens=quesLens)
    answers, ansLens = aBot.forwardDecode(
         inference='greedy', beamSize=beamSize)
    aBot.observe(round, ans=answers, ansLens=ansLens)
    qBot.observe(round, ans=answers, ansLens=ansLens)
    
    # Printing
    print("Q%d: "%(round+1), to_str_gt(questions[0]))
    print("A%d: "%(round+1), to_str_gt(answers[0]))
    print('')


summ, summLens = qBot.predictSummary(inference='greedy', beamSize=5)
print(Rouge().get_scores(summary, to_str_gt(summ[0]), avg=True))
print('----------------------------------------')
print("Summary: ", to_str_gt(summ[0]))

Q1:  is ? <END> <END> <END>
A1:  yes <END>

Q2:  is ? ? ? ? <END>
A2:  yes <END>

Q3:  is ? ? ? ? <END>
A3:  yes <END>

Q4:  is ? ? ? ? <END>
A4:  yes <END>

Q5:  is ? ? ? ? <END>
A5:  yes <END>

Q6:  is ? ? ? ? <END>
A6:  yes <END>

Q7:  is ? ? ? ? <END>
A7:  yes <END>

Q8:  is ? ? ? ? <END>
A8:  yes <END>

Q9:  is ? ? ? ? <END>
A9:  yes <END>

Q10:  is ? ? ? ? <END>
A10:  yes <END>

{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}
----------------------------------------
Summary:  number cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah cheetah 

In [150]:
Rouge().get_scores(summary, to_str_gt(summ[0]), avg=True)

{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}

In [49]:
Rouge().get_scores(to_str_gt(summ[0]), docs, avg=True)

{'rouge-1': {'r': 0.010857763300760043,
  'p': 0.2777777777777778,
  'f': 0.0208986408642473},
 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-l': {'r': 0.009771986970684038, 'p': 0.25, 'f': 0.018808776705417623}}

In [52]:


for word in summary.split(' '):
    if word in to_str_gt(summ[0]).split(' '):
        print(word)

well
